In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from datetime import datetime
import joblib

In [2]:
df = pd.read_csv('E:/GUVI Projects/MSCS/GUIDE_Train.csv')

In [3]:
df.head()

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [4]:
def change_dtype(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type == 'int64':
            df[col] = df[col].astype('int32')
        elif col_type == 'float64':
            df[col] = df[col].astype('float32')
        elif col_type == 'object':
            df[col] = df[col].astype('category')
    return df

In [5]:
df = change_dtype(df)

In [6]:
target_col = 'IncidentGrade'

In [7]:
def desc_na(data):
    total=data.isnull().sum().sort_values(ascending=False)
    percent=(data.isnull().sum()/data.isnull().count()*100).sort_values(ascending=False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [8]:
desc_na(df)

,Total,Percent
ResourceType,9509762,99.925658
ActionGrouped,9460773,99.410897
ActionGranular,9460773,99.410897
ThreatFamily,9441956,99.213173
EmailClusterId,9420025,98.982729
AntispamDirection,9339535,98.136965
Roles,9298686,97.707736
SuspicionLevel,8072708,84.825536
LastVerdict,7282572,76.523030
MitreTechniques,5468386,57.460120


In [9]:
def remove_na(df):
    missing_percent  = df.isna().sum() / len(df) *100
    missing_percent = missing_percent[missing_percent > 0]
    to_be_removed = missing_percent[missing_percent > 50].index.tolist()
    df.drop(columns=to_be_removed , axis=1,inplace=True)
    return df

In [10]:
df = remove_na(df)

In [11]:
categorical_columns = df.select_dtypes(include=['object','category']).columns

mode_val = df['IncidentGrade'].mode()[0]
print(mode_val)
df['IncidentGrade'] = df['IncidentGrade'].fillna(mode_val)

BenignPositive


In [12]:
def remove_outliers(df, iqr_multiplier=1.5):
    numeric_features = df.select_dtypes(include=['int32', 'int16', 'float32', 'int64', 'float64'])
    for col in numeric_features.columns:
        if df[col].nunique() > 10:
            Q1 = numeric_features[col].quantile(0.25)
            Q3 = numeric_features[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - iqr_multiplier * IQR
            upper_bound = Q3 + iqr_multiplier * IQR   
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

df = remove_outliers(df, iqr_multiplier=1.5) 

In [13]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['Hour'] = df['Timestamp'].dt.hour

In [14]:
df.drop(columns=['Timestamp'], inplace=True)

In [15]:
id_columns = ['Id', 'OrgId', 'IncidentId', 'AlertId', 'DetectorId', 'DeviceId', 'Sha256', 
              'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 
              'DeviceName', 'NetworkMessageId',  'RegistryKey', 'RegistryValueName', 
              'RegistryValueData', 'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 
              'FileName', 'FolderPath', 'ResourceIdName','OSFamily', 'OSVersion', 'City','CountryCode', 'State']

numeric_features = df.select_dtypes(include=['int32', 'int16', 'float32', 'int64']).drop(columns=id_columns, errors='ignore')

In [16]:
scaler = MinMaxScaler()
df[numeric_features.columns] = scaler.fit_transform(numeric_features)

In [17]:
label_encoder = LabelEncoder()
categorical_columns = ['Category', 'IncidentGrade', 'EntityType', 'EvidenceRole']
encod_dict = {}
for col in categorical_columns:
    if col in df.columns:
        df[col] = label_encoder.fit_transform(df[col].astype(str))
        label_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
        encod_dict[col] = label_mapping

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df.to_csv('E:/GUVI Projects/MSCS/scalar_train.csv',index=False)

In [20]:
df.head()

,Id,OrgId,IncidentId,AlertId,DetectorId,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,...,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City,Year,Month,DayOfWeek,Hour
24,3028,75,478,939479,4,0.006834,5,2,0,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.000000,0.608696
27,1813,0,211,160158,0,0.000000,5,2,0,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.166667,0.347826
42,1455,88,26726,1114326,58,0.097950,4,1,6,0,...,3586,5,66,242,1445,10630,1.0,0.454545,0.000000,0.043478
49,1731,0,375,169963,0,0.000000,5,2,0,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.000000,0.217391
50,4097,108,144844,118616,6,0.011390,5,2,5,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.500000,0.652174


In [21]:
df = pd.read_csv('E:/GUVI Projects/MSCS/GUIDE_Test.csv')


C:\Users\vivsk\AppData\Local\Temp\ipykernel_10520\2841289071.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('E:/GUVI Projects/MSCS/GUIDE_Test.csv')


In [39]:
df.head()

,Id,OrgId,IncidentId,AlertId,DetectorId,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,...,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City,Year,Month,DayOfWeek,Hour
3,5148,222,15294,917686,0,0.000000,5,1,1,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.333333,0.521739
27,2405,5,188,938345,19,0.063559,5,2,1,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.166667,0.782609
36,3880,72,153,897444,4,0.012712,5,2,0,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.333333,0.608696
42,2901,205,15974,148173,1,0.004237,5,2,5,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.833333,1.000000
53,2149,90,205,883099,0,0.000000,5,2,1,1,...,3586,5,66,242,1445,10630,1.0,0.454545,0.000000,0.869565


In [22]:
def change_dtype(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type == 'int64':
            df[col] = df[col].astype('int32')
        elif col_type == 'float64':
            df[col] = df[col].astype('float32')
        elif col_type == 'object':
            df[col] = df[col].astype('category')
    return df

In [23]:
df = change_dtype(df)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4147992 entries, 0 to 4147991
Data columns (total 46 columns):
 #   Column              Dtype   
---  ------              -----   
 0   Id                  int32   
 1   OrgId               int32   
 2   IncidentId          int32   
 3   AlertId             int32   
 4   Timestamp           category
 5   DetectorId          int32   
 6   AlertTitle          int32   
 7   Category            category
 8   MitreTechniques     category
 9   IncidentGrade       category
 10  ActionGrouped       category
 11  ActionGranular      category
 12  EntityType          category
 13  EvidenceRole        category
 14  DeviceId            int32   
 15  Sha256              int32   
 16  IpAddress           int32   
 17  Url                 int32   
 18  AccountSid          int32   
 19  AccountUpn          int32   
 20  AccountObjectId     int32   
 21  AccountName         int32   
 22  DeviceName          int32   
 23  NetworkMessageId    int32   
 24

In [25]:
def remove_na(df):
    missing_percent  = df.isna().sum() / len(df) *100
    missing_percent = missing_percent[missing_percent > 0]
    to_be_removed = missing_percent[missing_percent > 50].index.tolist()
    df.drop(columns=to_be_removed , axis=1,inplace=True)
    return df

In [26]:
df = remove_na(df)

In [27]:
def outliers(df, iqr_multiplier=1.5):
    numeric_features = df.select_dtypes(include=['int32', 'int16', 'float32', 'int64', 'float64'])
    for col in numeric_features.columns:
        Q1 = numeric_features[col].quantile(0.25)
        Q3 = numeric_features[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR
            
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

df = outliers(df, iqr_multiplier=1.5)

In [28]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['Hour'] = df['Timestamp'].dt.hour
df.drop(columns=['Timestamp','Usage'], inplace=True)

In [33]:
id_columns = ['Id', 'OrgId', 'IncidentId', 'AlertId', 'DetectorId', 'DeviceId', 'Sha256', 
              'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 
              'DeviceName', 'NetworkMessageId',  'RegistryKey', 'RegistryValueName', 
              'RegistryValueData', 'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 
              'FileName', 'FolderPath', 'ResourceIdName','OSFamily','OSVersion', 'City','CountryCode', 'State']

numeric_features = df.select_dtypes(include=['int32', 'int16', 'float32', 'int64']).drop(columns=id_columns, errors='ignore')

In [34]:
scaler = MinMaxScaler()
df[numeric_features.columns] = scaler.fit_transform(numeric_features)

In [35]:
label_encoder = LabelEncoder()
categorical_columns = ['Category', 'IncidentGrade', 'EntityType', 'EvidenceRole']

for col in categorical_columns:
    if col in df.columns:
        df[col] = label_encoder.fit_transform(df[col].astype(str))

In [36]:
df.drop_duplicates(inplace=True)

In [37]:
df.to_csv('E:/GUVI Projects/MSCS/scalar_test.csv',index=False)

In [38]:
df.columns

Index(['Id', 'OrgId', 'IncidentId', 'AlertId', 'DetectorId', 'AlertTitle',
       'Category', 'IncidentGrade', 'EntityType', 'EvidenceRole', 'DeviceId',
       'Sha256', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn',
       'AccountObjectId', 'AccountName', 'DeviceName', 'NetworkMessageId',
       'RegistryKey', 'RegistryValueName', 'RegistryValueData',
       'ApplicationId', 'ApplicationName', 'OAuthApplicationId', 'FileName',
       'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode',
       'State', 'City', 'Year', 'Month', 'DayOfWeek', 'Hour'],
      dtype='object')